<h1>Community Detection</h1>

In [1]:
import os
import networkx as nx
import numpy as np
import pandas as pd
import csv
from networkx.algorithms.community.centrality import girvan_newman
from sklearn.cluster import SpectralClustering
from sklearn.metrics import normalized_mutual_info_score
from networkx.algorithms.clique import find_cliques
from networkx import edge_betweenness_centrality as betweenness
from collections import Counter
import math

In [2]:
os.getcwd()

'c:\\Users\\ASUS\\Desktop\\MSc\\MSBD 5002\\hdpns_20315566_MSBDT5002_final\\Q1\\Q1_code'

In [3]:
email_path = os.path.join(os.getcwd(),"email-Eu-core.txt")
department_label = os.path.join(os.getcwd(),"email-Eu-core-department-labels.txt")

In [4]:
def split_data_email(x):
    x["Source"] = int(x[0].split(" ")[0])
    x["Target"] = int(x[0].split(" ")[1])
    return x
def split_data_department(x):
    x["Node Id"] = int(x[0].split(" ")[0])
    x["Department Id"] = int(x[0].split(" ")[1])
    return x

In [5]:
emaildf = pd.read_csv(email_path,header=None).apply(lambda x: split_data_email(x),axis = 1)[["Source","Target"]]
departmentdf = pd.read_csv(department_label,header=None).apply(lambda x: split_data_department(x),axis = 1)[["Node Id","Department Id"]]

In [6]:
emaildf

,Source,Target
0,0,1
1,2,3
2,2,4
3,5,6
4,5,7
...,...,...
25566,420,143
25567,174,859
25568,440,460
25569,52,786


In [7]:
departmentdf

,Node Id,Department Id
0,0,1
1,1,1
2,2,21
3,3,21
4,4,21
...,...,...
1000,1000,4
1001,1001,21
1002,1002,1
1003,1003,6


In [8]:
nodes = np.unique(departmentdf["Node Id"])
graph_numpy = np.zeros(shape = (len(nodes),len(nodes)))
for i,j in zip(emaildf["Source"],emaildf["Target"]):
    graph_numpy[int(i),int(j)] = graph_numpy[int(i),int(j)] + 1

<h2> Spectral Clustering </h2>

In [9]:
#in the spectral clustering API of sklearn, n_clusters is the number of clusters required,
#n_components is the number of eigenvectors to use for spectral embedding. 
#n_init is the number of times the k-means clustering algorithm will be run 
#after spectral clustering. We vary the n_components to find which gives the best NMI score
#As per sklearn, a high value of NMI close to 1 indicates good correlation in results

#Note That the cluster labels will not be same as the ground truth as Spectral Clustering randomly
#assigns labels. However given entropy calculations for NMI consider the counts of labels, this 
#doesnt have an effect

#also note that a graph is made using numpy array as shown in below code
NMI_scores = list()
for n_comp in range(10,40):
    nodes = np.unique(departmentdf["Node Id"])
    graph_numpy = np.zeros(shape = (len(nodes),len(nodes)))
    for i,j in zip(emaildf["Source"],emaildf["Target"]):
        graph_numpy[int(i),int(j)] = graph_numpy[int(i),int(j)] + 1
    sc = SpectralClustering(42, affinity='precomputed', n_init=100,n_components=n_comp)
    sc.fit(graph_numpy)
    nmi = normalized_mutual_info_score(departmentdf["Department Id"],sc.labels_)
    NMI_scores.append((n_comp,nmi))


C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  return f(*args, **kwargs)
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  return f(*args, **kwargs)
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral e

In [11]:
optimal_number_of_n_components = sorted(NMI_scores,key = lambda x:x[1],reverse = True)[0]
optimal_number_of_n_components

(22, 0.653758234321499)

Optimal number of eigen vectors to use for Spectral Clustering is found to be 22. 

In [16]:
nodes = np.unique(departmentdf["Node Id"])
graph_numpy = np.zeros(shape = (len(nodes),len(nodes)))
for i,j in zip(emaildf["Source"],emaildf["Target"]):
    graph_numpy[int(i),int(j)] = graph_numpy[int(i),int(j)] + 1
sc = SpectralClustering(42, affinity='precomputed', n_init=100,n_components=22)
sc.fit(graph_numpy)

C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:63: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  return f(*args, **kwargs)
C:\Users\ASUS\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


SpectralClustering(affinity='precomputed', n_clusters=42, n_components=22,
                   n_init=100)

In [17]:
result = sc.labels_

In [20]:
submission = pd.DataFrame({"node id":list(range(0,1005)),"department id":result})
submission

,node id,department id
0,0,31
1,1,31
2,2,27
3,3,27
4,4,27
...,...,...
1000,1000,28
1001,1001,27
1002,1002,38
1003,1003,14


In [21]:
submission.to_csv('Q1_communities.csv')